In [1]:
import pandas as pd
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.io as pio
from plotly.colors import n_colors
import plotly.express as px
import plotly.figure_factory as ff
import datetime as dt
from datetime import date, timedelta, datetime
from mpg_refresh import mpg_data_creator, insight_creator
df = pd.read_csv('car_mpg_data.csv')
df = mpg_data_creator(df)


In [2]:
# Format dollars to contain 2 decimal places
# Change line colors
# Make plot bigger
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date'],
                                y=df['gal_cost'],
                                # yaxis=dict(range=[1.2, 2.8]),
                                mode='lines',
                                hovertemplate='$' +  df['gal_cost'].apply(lambda x: '{:.2f}'.format(x))+ ' on ' + df['date'].dt.strftime('%b %d %Y'),
                                name='Gallon cost',
                                line=dict(color="#1A4D94")
))
fig.add_trace(go.Scatter(x=df['date'],
                                y=df['gal_cost'].rolling(window=5).mean(),
                                # yaxis=dict(range=[1.2, 2.8]),
                                mode='lines',
                                hovertemplate='$' + round(df['gal_cost'].rolling(window=5).mean(),2).apply(lambda x: '{:.2f}'.format(x)) + ' on ' + df['date'].dt.strftime('%b %d %Y'),
                                name='Moving average',
                                line=dict(color="#5C7DAA")
))

fig.update_layout(
    showlegend=False,
    width=1000,
    height=400,
    updatemenus=[
        dict(
            type = "buttons",
            buttons=list([
                dict(
                    args = [{'visible': [True, False]},
                            {'title': 'Gallon Cost'}],
                    label="Gallon cost",
                    method="restyle"
                ),
                dict(
                    args = [{'visible': [False, True]},
                            {'title': 'Moving average'}],
                    label="Moving average",
                    method="restyle"
                ),
                dict(
                    args = [{'visible': [True, True]}],
                    label="Both",
                    method="restyle"
                )
            ]),
            direction='down',
            showactive=True,
            x=1.05,
            xanchor="left",
            yanchor="top"
        ),
    ],
    plot_bgcolor='#cccccc',
    title=dict(
        text='Cost of a gallon of gas over time',
        font=dict(
            size=24,
            color='#000000'
        ),
        x=.5
    ),
    xaxis=dict(
        title='Date',
        range=[df['date'].min() - dt.timedelta(days=5), df['date'].max() + dt.timedelta(days=5)]
    ),
    yaxis=dict(
        title='Cost',
        range=[df['gal_cost'].min() - .2, df['gal_cost'].max() + .2]
    )
)

fig.show()

In [3]:
# Change line colors
# Make figure bigger
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['date'],
                                y=df['mpg'],
                                mode='lines',
                                hovertemplate=df['mpg'].astype(str)+ ' on ' + df['date'].dt.strftime('%b %d %Y'),
                                name='MPG',
                                line=dict(color="#1A4D94")
))
fig.add_trace(go.Scatter(x=df['date'],
                                y=df['mpg'].rolling(window=5).mean(),
                                mode='lines',
                                hovertemplate=round(df['mpg'].rolling(window=5).mean(),2).astype(str) + ' on ' + df['date'].dt.strftime('%b %d %Y'),
                                name='Moving average',
                                line=dict(color="#5C7DAA")
))

fig.update_layout(
    showlegend=False,
    plot_bgcolor='#cccccc',
    width=1000,
    height=400,
    updatemenus=[
        dict(
            type = "buttons",
            buttons=list([
                dict(
                    args=[dict(visible=[True, False]),
                            dict(title='MPG')],
                    label="MPG",
                    method="restyle"
                ),
                dict(
                    args=[dict(visible=[False, True]),
                            dict(title='Moving average')],
                    label="Moving average",
                    method="restyle"
                ),
                dict(
                    args = [dict(visible=[True, True])],
                    label="Both",
                    method="restyle"
                )
            ]),
            direction='down',
            showactive=True,
            x=1.05,
            xanchor="left",
            yanchor="top"
        ),
    ],
    title=dict(
        text='Miles per gallon over time',
        font=dict(
            size=24,
            color='#000000'
        ),
        x=.5
    ),
    xaxis=dict(
        title='Date',
        range=[df['date'].min() - dt.timedelta(days=5), df['date'].max() + dt.timedelta(days=5)]
    ),
    yaxis=dict(
        title='MPG',
        range=[df['mpg'].min() - 1, df['mpg'].max() + 1]
    )
)

fig.show()

In [4]:
# moving average chart for car mpg and cost for gallon of gas
# also chart with moving average for mpg but a mark on chart gal cost


In [5]:
# mpg vs. miles driven scatter plot
# colored by month

In [6]:
# $ per mile vs. miles scatter plot
# colored by month

In [7]:
# High/low temp over a year with mpg